# Projeto 1 - Ciência dos Dados

Nome: Letícia Coêlho Barbosa

Nome: Matheus Silva Melo de Oliveira

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [981]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [982]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\mathe_vz6v46f\OneDrive\Documentos\Arquivos INSPER 2° SEMESTRE\C-Dados\TheMandalorian


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [983]:
filename = 'mandalorian.xlsx'

In [984]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,executivo da disney defendeu demissão de gina ...,0
1,@daredvevil nao seriooo melhor dia 😭 volta the...,1
2,preciso terminar mandalorian logo,1
3,toda vez que eu vejo o baby yoda usando a forç...,0
4,jon fraveau eu te amo obrigado por fazer the m...,1


In [985]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,the mandalorian é uma série com dinheiro e tec...,1
1,agora comecei mandalorian confiando novamente ...,1
2,@fepazo1 @gabriel_ns1999 @joelmsimmons @jurand...,1
3,eu assisti 3 minutos de mandalorian e emocionei,1
4,rt @anakinwar: já ta ...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [986]:
# !pip install emoji

In [987]:
import re 
import emoji
from emoji import UNICODE_EMOJI

train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [988]:
#Limpando os tweets retirando pontos desnecessarios
def limpeza(tweet):
    punctuation = '[!-.:?;\n|]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern,'', tweet)
    return text_subbed


In [989]:
def retira_emoji(content: str):
        modified = re.sub(emoji.get_emoji_regexp(), r"", content)
        modified = re.sub(':[^>]+:', '', modified)
        modified = re.sub('<[^>]+>', '', modified)
        return modified.strip() 


In [990]:
def lower(tweet):
    return tweet.lower()

In [991]:
#Aplicando limpeza necessária, e quebrando em listas
def concatenar(tweet):
    
    #Tornando os emojis em strings descritivas
    emojis=re.findall(emoji.get_emoji_regexp(),tweet)
    #Algus emojis não possuem transcrição para português
    for i,emoji1 in enumerate(emojis):
            if emoji1 in UNICODE_EMOJI['pt']:
                emojis[i]=UNICODE_EMOJI['pt'][emoji1].replace(':',',')
            else:
                emojis[i]=UNICODE_EMOJI['en'][emoji1].replace(':',',')

                
    g=retira_emoji(tweet)
    g=lower(g)
    g=g.split()
    
    for i,word in enumerate(g):
        
        if 'rt' in word or 'https' in word or '@' in word:
            g[i]=g[i].replace(word,'null')
        
    g=[x for x in g if x!='null']
    k=','.join(g)
    m=','.join(emojis)
    return k+','+m


In [992]:
train['Clean']=train['Treinamento'].apply(limpeza)
train['Clean']=train['Clean'].apply(concatenar)

test['Clean']=test['Teste'].apply(limpeza)
test['Clean']=test['Clean'].apply(concatenar)

train_relevantes=train[train['Relevancia']==1]
train_irrelevantes=train[train['Relevancia']==0]

In [993]:
train.Treinamento[23]


'obrigada aos envolvidos por mandalorian, é simplesmente a melhor coisa que eu ja assisti na minha vida. meu lado fã de star wars esta em êxtase nesse exato momento'

In [994]:
train.Clean[36]

'mandalorian,pobre,não,tem,dinheiro,para,o,peitoral,,rosto_deprimido,'

In [995]:
#Criando texto com palavras relevantes:

texto_r=''

for palavra in train_relevantes['Clean']:
    texto_r+=str(palavra)

In [996]:
#Criando texto com palavras irrelevantes:
texto_i=''

for palavra in train_irrelevantes['Clean']:
    texto_i+=str(palavra)


In [997]:
#Tranformando cada texto em uma Série de palavras:
palavras_r=pd.Series(texto_r.split(','))
palavras_i=pd.Series(texto_i.split(','))
palavras_r

0                                 nao
1                             seriooo
2                              melhor
3                                 dia
4                               volta
                    ...              
4719                              the
4720                      mandalorian
4721                                 
4722    mãos_juntas_pele_morena_clara
4723                                 
Length: 4724, dtype: object

In [998]:
palavras_r

0                                 nao
1                             seriooo
2                              melhor
3                                 dia
4                               volta
                    ...              
4719                              the
4720                      mandalorian
4721                                 
4722    mãos_juntas_pele_morena_clara
4723                                 
Length: 4724, dtype: object

In [999]:
palavras_i

0       executivo
1              da
2          disney
3        defendeu
4        demissão
          ...    
3956          ver
3957            o
3958        pedro
3959       pascal
3960             
Length: 3961, dtype: object

In [1000]:
# Frequência Relativa das palavras relevantes:

rel_relevantes=palavras_r.value_counts(normalize=True)

In [1001]:
rel_relevantes.head(12)

mandalorian    0.057155
               0.041702
de             0.040644
the            0.031753
é              0.020322
a              0.020110
que            0.019898
o              0.019898
e              0.019687
eu             0.017570
da             0.010584
mas            0.008891
dtype: float64

In [1002]:
lista_r=['mandalorian','de','the','é','que','a','e','o','eu','da','mas','com']
lista_r

for i in lista_r:
    print(rel_relevantes[i])

0.0571549534292972
0.04064352243861134
0.03175275190516511
0.02032176121930567
0.01989839119390347
0.02011007620660457
0.01968670618120237
0.01989839119390347
0.017569856054191365
0.010584250635055038
0.008890770533446233
0.007620660457239628


In [1003]:
# Frequência Relativa das palavras irrelevantes:
rel_irrelevantes=palavras_i.value_counts(normalize=True)

In [1004]:
rel_irrelevantes.head(18)

mandalorian    0.049735
de             0.037617
o              0.023984
the            0.023731
e              0.023479
a              0.020197
que            0.018177
eu             0.015148
               0.014138
do             0.012876
é              0.011866
da             0.010856
em             0.010098
um             0.009089
não            0.009089
no             0.009089
pra            0.008584
com            0.007574
dtype: float64

In [1005]:
lista_i=['mandalorian','de','the','e','o','a','que','do','é','eu','da','em','no','um','pra','não','com']

for i in lista_i:
    print(rel_irrelevantes[i])

0.049734915425397624
0.037616763443574855
0.02373138096440293
0.02347891946478162
0.023983842464024237
0.02019691996970462
0.018177227972734158
0.012875536480686695
0.011865690482201464
0.015147689977278465
0.010855844483716234
0.01009845998485231
0.009088613986367079
0.009088613986367079
0.008583690987124463
0.009088613986367079
0.007573844988639233


In [1006]:
#Serie de todas as palavras:

palavras_i=palavras_i.tolist()
palavras_r=palavras_r.tolist()

total=palavras_i+palavras_r

total=pd.Series(total)
total

0                           executivo
1                                  da
2                              disney
3                            defendeu
4                            demissão
                    ...              
8680                              the
8681                      mandalorian
8682                                 
8683    mãos_juntas_pele_morena_clara
8684                                 
Length: 8685, dtype: object

In [1007]:
#Frequência das palvras no total:

rel_total=total.value_counts(True)

### Montando Probabilidades

In [1008]:
#Probabilidade tweet ser relevante ou irrelevante:

P_r=len(palavras_r)/len(total)
P_i=len(palavras_i)/len(total)

In [1009]:
lista_r=['mandalorian','de','the','é','que','a','e','o','eu','da','mas','com']
lista_i=['mandalorian','de',' ','the','e','o','a','que','do','é','eu','da','em','no','um','pra','não','com']

In [1038]:
def P_tweet_r (tweet):
    
    c=1
    
    for palavra in tweet.split(','):
        if(palavra in palavras_r) and (palavra not in lista_r) :
            
            c*=rel_relevantes[palavra]
        elif palavra in rel_total:
             c*=rel_total[palavra]
        else:
            continue

    return c

In [1039]:
def P_tweet_i (tweet):
    
    c=1
    
    for palavra in tweet.split(','):
        if(palavra in palavras_i) and (palavra not in lista_i):
            c*=rel_irrelevantes[palavra]
        elif palavra in rel_total:
            c*=rel_total[palavra]
        else:
            continue
        
    return c

In [1040]:
P_tweet_r('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars')

1.1011617139115327e-47

In [1041]:
P_tweet_i('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars')

6.751332335548268e-51

In [1042]:
def Naive_Bayes (tweet):
    if(P_tweet_r(tweet)*P_r > P_tweet_i(tweet)*P_i):
        return 1
    else:
        return 0

In [1043]:
train['Classificador']=train.Clean.apply(Naive_Bayes)

In [1044]:
x=train.loc[(train['Classificador']==1)&(train['Relevancia']==1),:].shape[0]
y=train.loc[(train['Classificador']==0)&(train['Relevancia']==0),:].shape[0]
(x+y)/train.shape[0]

0.924

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [1050]:
test['Classificador']=test.Clean.apply(Naive_Bayes)

,Teste,Relevancia,Clean,Classificador
0,the mandalorian é uma série com dinheiro e tec...,1,"the,mandalorian,é,uma,série,com,dinheiro,e,tec...",1
1,agora comecei mandalorian confiando novamente ...,1,"agora,comecei,mandalorian,confiando,novamente,...",1
2,@fepazo1 @gabriel_ns1999 @joelmsimmons @jurand...,1,"the,mandalorian,ousou,ao,trazer,personagens,do...",1
3,eu assisti 3 minutos de mandalorian e emocionei,1,"eu,assisti,3,minutos,de,mandalorian,e,emocionei,",1
4,rt @anakinwar: já ta ...,0,"já,ta,mas,mãechorando,aquele,final,dednv,garot...",1
...,...,...,...,...
220,"tô com saudade de the mandalorian, acho que vo...",1,"tô,com,saudade,de,the,mandalorian,acho,que,vou...",1
221,meu deus pedro pascal é o mandalorian 😱😱😱😱😱😱😱,1,"meu,deus,pedro,pascal,é,o,mandalorian,,rosto_g...",1
222,@39marukin @ghostjacobs tem. assisti lá.\nagor...,1,"tem,assisti,láagora,finalmente,vou,começar,man...",1
223,@ip_mandalorian @tmarretamma ficou parecendo o...,1,"ficou,parecendo,o,pequenino,do,filme,,rolando_...",1


In [1047]:
x=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
y=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]
(x+y)/test.shape[0]

0.56

In [1052]:
# fp=test.loc[(test['Classificador']==0)&(test['Relevancia']==1),:].shape[0]
# tp=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
# fn=test.loc[(test['Classificador']==1)&(test['Relevancia']==0),:].shape[0]
# tn=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]

# test['Classificador']=
pd.crosstab(test.Relevancia,test.Classificador,normalize=True)

Classificador,0,1
Relevancia,,
0,0.155556,0.391111
1,0.048889,0.404444


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**